# Machine learning

In [240]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.signal as sg
import serial
import time
import random

matplotlib.rcParams['figure.figsize'] = [10, 3]

In [241]:
SAMPLE_NUMS = 260  # sec
SAMPLING_RATE = 80.0  # Hz
GYRO_RESOLUTION = 250.0 / 32768.0
ACCEL_RESOLUTION = 2.0 / 32768.0

FILE0 = './20180821_walking_straight.csv'
FILE1 = './20180821_sitting_down.csv'
FILE2 = './20180821_turning_left.csv'
#FILE3 = './20180821_turning_right.csv'

TIME_INTERVAL = 260.0 / SAMPLING_RATE  # sec

MEASUREMENTS = 8
RECORDS = 260

## Preparing data set for training CNN

In [242]:
gres = lambda v: v * GYRO_RESOLUTION
ares = lambda v: v * ACCEL_RESOLUTION
to_time = lambda v: v / SAMPLING_RATE

def conv(df):
    df[['gx', 'gy', 'gz']] = df[['gx', 'gy', 'gz']].apply(gres)
    df[['ax', 'ay', 'az']] = df[['ax', 'ay', 'az']].apply(ares)
    CUTOFF = 10.0
    b, a = sg.butter(5, CUTOFF/SAMPLING_RATE, btype='low')
    df[['ax', 'ay', 'az']] = df[['ax', 'ay', 'az']].apply(lambda row: sg.lfilter(b, a, row))
    CUTOFF = 10.0
    b, a = sg.butter(5, CUTOFF/SAMPLING_RATE, btype='low')
    df[['gx', 'gy', 'gz']] = df[['gx', 'gy', 'gz']].apply(lambda row: sg.lfilter(b, a, row))
    df[['cnt']] = df[['cnt']].apply(to_time)
    df.set_index('cnt', drop=True, inplace=True)
    # measurements = df.tail(1).iloc[0,0] + 1
    # return measurements

In [243]:
df0 = pd.read_csv(FILE0, dtype=np.int16)
df1 = pd.read_csv(FILE1, dtype=np.int16)
df2 = pd.read_csv(FILE2, dtype=np.int16)
#df3 = pd.read_csv(FILE3, dtype=np.int16)

conv(df0)
conv(df1)
conv(df2)
#conv(df3)

In [244]:
df0['label'] = np.full(RECORDS*MEASUREMENTS, 0)
df1['label'] = np.full(RECORDS*MEASUREMENTS, 1)
df2['label'] = np.full(RECORDS*MEASUREMENTS, 2)
#df3['label'] = np.full(RECORDS*MEASUREMENTS, 2)

df_set, df0set, df1set, df2set = [], [], [], []
#df_set, df0set, df1set = [], [], []
for i in range(MEASUREMENTS):
    df0set.append([df0[df0['id'] == i], [1,0,0]])
    df1set.append([df1[df1['id'] == i], [0,1,0]])
    df2set.append([df2[df2['id'] == i], [0,0,1]])
#   df2set.append([df2[df2['id'] == i], [0,0,0,1]])

df_set.extend(df0set)
df_set.extend(df1set)
df_set.extend(df2set)
#df_set.extend(df3set)

random.shuffle(df_set)

In [245]:
train_x = []
train_t = []
for df, label in df_set:
    train_x.append(df['az'].values)
    train_t.append(label)
#train_x

## TensorFlow

In [246]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20180821)
tf.set_random_seed(20180821)

In [247]:
num_units = 16

x = tf.placeholder(tf.float32, [None, 260])

w1 = tf.Variable(tf.truncated_normal([260, num_units]))
b1 = tf.Variable(tf.zeros([num_units]))
hidden1 = tf.nn.tanh(tf.matmul(x, w1) + b1)

w0 = tf.Variable(tf.zeros([num_units, 3]))
b0 = tf.Variable(tf.zeros([3]))
p = tf.nn.softmax(tf.matmul(hidden1, w0) + b0)

In [248]:
t = tf.placeholder(tf.float32, [None, 3])
loss = -tf.reduce_sum(t * tf.log(p))
train_step = tf.train.AdamOptimizer().minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [249]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [250]:
i = 0
for _ in range(2000):
    i += 1
    sess.run(train_step, feed_dict={x:train_x, t:train_t})
    if i % 100 == 0:
        loss_val, acc_val = sess.run([loss, accuracy], feed_dict={x:train_x, t: train_t})
        print('Step: {}, Loss: {}, Accuracy: {}'.format(i, loss_val, acc_val))

Step: 100, Loss: 23.68749237060547, Accuracy: 0.75
Step: 200, Loss: 20.612253189086914, Accuracy: 0.7083333134651184
Step: 300, Loss: 18.428131103515625, Accuracy: 0.75
Step: 400, Loss: 16.851245880126953, Accuracy: 0.75
Step: 500, Loss: 15.73366928100586, Accuracy: 0.75
Step: 600, Loss: 14.938302993774414, Accuracy: 0.75
Step: 700, Loss: 14.356730461120605, Accuracy: 0.75
Step: 800, Loss: 13.91321849822998, Accuracy: 0.75
Step: 900, Loss: 13.560996055603027, Accuracy: 0.75
Step: 1000, Loss: 13.270149230957031, Accuracy: 0.75
Step: 1100, Loss: 13.019122123718262, Accuracy: 0.75
Step: 1200, Loss: 12.79128646850586, Accuracy: 0.75
Step: 1300, Loss: 12.561667442321777, Accuracy: 0.75
Step: 1400, Loss: 12.314582824707031, Accuracy: 0.75
Step: 1500, Loss: 12.083556175231934, Accuracy: 0.75
Step: 1600, Loss: 11.867513656616211, Accuracy: 0.75
Step: 1700, Loss: 11.668277740478516, Accuracy: 0.75
Step: 1800, Loss: 11.373058319091797, Accuracy: 0.75
Step: 1900, Loss: 10.5880708694458, Accuracy:

In [251]:
p_test = sess.run(p, feed_dict={x:train_x})
p_test

array([[9.93604600e-01, 3.20562394e-04, 6.07495848e-03],
       [2.12222859e-01, 1.74142793e-01, 6.13634348e-01],
       [1.42434975e-02, 7.00233221e-01, 2.85523236e-01],
       [1.72671542e-01, 2.21190155e-01, 6.06138289e-01],
       [1.42391510e-02, 7.00285494e-01, 2.85475314e-01],
       [2.02333897e-01, 1.80070043e-01, 6.17596090e-01],
       [9.82537568e-01, 2.01773737e-03, 1.54446876e-02],
       [4.83544081e-01, 7.22209364e-02, 4.44234997e-01],
       [1.82776093e-01, 2.08776891e-01, 6.08447015e-01],
       [9.85287596e-04, 9.88924682e-01, 1.00899795e-02],
       [1.42700281e-02, 6.99984431e-01, 2.85745561e-01],
       [2.03174815e-01, 1.79234326e-01, 6.17590845e-01],
       [2.04539210e-01, 1.78357974e-01, 6.17102802e-01],
       [4.74380940e-01, 4.09808643e-02, 4.84638184e-01],
       [4.75971609e-01, 4.06070948e-02, 4.83421355e-01],
       [1.42227290e-02, 7.00434208e-01, 2.85343051e-01],
       [4.74064201e-01, 4.10547107e-02, 4.84880984e-01],
       [9.39849734e-01, 1.93093

In [252]:
train_t

[[1, 0, 0],
 [0, 0, 1],
 [0, 1, 0],
 [0, 0, 1],
 [0, 1, 0],
 [0, 0, 1],
 [1, 0, 0],
 [1, 0, 0],
 [0, 0, 1],
 [0, 1, 0],
 [0, 1, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [1, 0, 0],
 [0, 1, 0],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 0, 1],
 [0, 1, 0],
 [0, 1, 0],
 [0, 1, 0],
 [1, 0, 0]]

In [253]:
sess.close()